In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
#!apt install git-lfs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

# **CNN Dataset**

In [ ]:
#!pip install datasets
#!pip install scikit-learn
#!pip install transformers

In [ ]:
from datasets import load_dataset

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0')


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:


# Split the dataset into smaller portions (e.g. 1/10th of the original size)
train_size = int(0.1 * len(cnn_dataset['train']))
val_size = int(0.1 * len(cnn_dataset['validation']))
test_size = int(0.1 * len(cnn_dataset['test']))

train_data = cnn_dataset['train'].train_test_split(train_size=train_size)['train']
val_data = cnn_dataset['validation'].train_test_split(train_size=val_size)['train']
test_data = cnn_dataset['test'].train_test_split(train_size=test_size)['train']

cnn_dataset['train'] = train_data
cnn_dataset['validation'] = val_data
cnn_dataset['test'] = test_data

In [ ]:
def show_samples(dataset, num_samples=3, seed=42):
    sample = dataset["train"].shuffle(seed=seed).select(range(num_samples))
    for example in sample:
        print(f"\n'>> Highlights: {example['highlights']}'")
        print(f"'>> Article: {example['article']}'")


show_samples(cnn_dataset)

In [ ]:
cnn_dataset = cnn_dataset.filter(lambda x: len(x["highlights"].split()) > 2)


In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
max_input_length = 512
max_target_length = 30


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["article"],
        max_length=max_input_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["highlights"], max_length=max_target_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = cnn_dataset.map(preprocess_function, batched=True)
tokenized_datasets

Graph representing top 50 words of highlights from the training part of the original dataset

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

word_counts = Counter()
for item in tokenized_datasets["train"]:
    # Remove punctuation characters
    article = item["highlights"].translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(article.lower())
    tokens = [token for token in tokens if token not in stop_words]
    word_counts.update(tokens)

# Get the 50 most common words and their frequencies
top_words = word_counts.most_common(50)
bottom_words = word_counts.most_common()[:-51:-1]



In [ ]:
# Plot the most common words
plt.figure(figsize=(10, 5))
sns.set_style("darkgrid")
sns.barplot(x=[word[0] for word in top_words], y=[word[1] for word in top_words])
plt.xticks(rotation=90)
plt.title("Top 50 most common words")
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.show()

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(
    cnn_dataset["train"].column_names
)

In [ ]:
features = [tokenized_datasets["train"][i] for i in range(2)]
data_collator(features)

In [ ]:
tf_train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)
tf_eval_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_epochs = 8
num_train_steps = len(tf_train_dataset) * num_train_epochs
model_name = model_checkpoint.split("/")[-1]

optimizer, schedule = create_optimizer(
    init_lr=5.6e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)

model.compile(optimizer=optimizer)

# Train in mixed-precision float16
tf.keras.mixed_precision.set_global_policy("mixed_float16")

In [ ]:
from transformers.keras_callbacks import PushToHubCallback
model.fit(
    tf_train_dataset, validation_data=tf_eval_dataset, epochs=1
)

In [ ]:
import nltk

nltk.download("punkt")

In [ ]:
from tqdm import tqdm
import numpy as np
from nltk.tokenize import sent_tokenize

generation_data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=320
)

tf_generate_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    collate_fn=generation_data_collator,
    shuffle=False,
    batch_size=8,
    drop_remainder=True,
)


@tf.function(jit_compile=True)
def generate_with_xla(batch):
    return model.generate(
        input_ids=batch["input_ids"],
        attention_mask=batch["attention_mask"],
        max_new_tokens=32,
    )


all_preds = []
all_labels = []
for batch, labels in tqdm(tf_generate_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = labels.numpy()
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    all_preds.extend(decoded_preds)
    all_labels.extend(decoded_labels)

In [ ]:
all_preds

Graph representing top 50 words in the Predicted summaries:

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

word_counts_pred = Counter()
for item in all_preds:
    # Remove punctuation characters
    article = item.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(article.lower())
    tokens = [token for token in tokens if token not in stop_words]
    word_counts_pred.update(tokens)

# Get the 50 most common words and their frequencies
top_words_pred = word_counts_pred.most_common(50)
bottom_words_pred = word_counts_pred.most_common()[:-51:-1]

print(top_words_pred)

[]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Plot the most common words
plt.figure(figsize=(10, 5))
sns.set_style("darkgrid")
sns.barplot(x=[word[0] for word in top_words_pred[2:]], y=[word[1] for word in top_words_pred[2:]])
plt.xticks(rotation=90)
plt.title("Top 50 most common words")
plt.xlabel("Words")
plt.ylabel("Frequency")
plt.show()

NameError: ignored

<Figure size 1000x500 with 0 Axes>

In [ ]:
all_labels

In [ ]:
!pip install rouge_score

In [ ]:
import evaluate

rouge_score = evaluate.load("rouge")

In [ ]:
result = rouge_score.compute(
    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
)
result = {key: value * 100 for key, value in result.items()}
{k: round(v, 4) for k, v in result.items()}

In [ ]:
from transformers import pipeline

hub_model_id = "huggingface-course/mt5-small-finetuned-amazon-en-es"
summarizer = pipeline("summarization", model=model)

In [ ]:
# performance comparison
# Convert the tokenized labels back to text
from tqdm import tqdm
def decode_label(tokenizer, labels):
    decoded_labels = []
    for label in labels:
        decoded_label = tokenizer.decode(label, skip_special_tokens=True)
        decoded_labels.append(decoded_label)
    return decoded_labels

# Generate predictions for the validation dataset
all_preds = []
for batch in tqdm(tf_eval_dataset):
    predictions = generate_with_xla(batch)
    decoded_preds = decode_label(tokenizer, predictions)
    all_preds.extend(decoded_preds)

# Get the actual labels for the validation dataset
all_labels = []
for item in tqdm(tokenized_datasets["validation"]):
    label = tokenizer.decode(item["labels"], skip_special_tokens=True)
    all_labels.append(label)

# Print some examples of actual and predicted summaries
for i in range(50):
    print(f"\nExample {i+1}:")
    print(f"\tActual summary: {all_labels[i]}")
    print(f"\tPredicted summary: {all_preds[i]}")

import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))

# Pre-training dataset
word_counts_pre = Counter()
for item in tokenized_datasets["train"]:
    # Remove punctuation characters
    article = item["highlights"].translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(article.lower())
    tokens = [token for token in tokens if token not in stop_words]
    word_counts_pre.update(tokens)

# Get the 50 most common words and their frequencies
top_words_pre = word_counts_pre.most_common(50)

# Post-training dataset
word_counts_post = Counter()
for item in tokenized_datasets["test"]:
    # Remove punctuation characters
    article = item["highlights"].translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(article.lower())
    tokens = [token for token in tokens if token not in stop_words]
    word_counts_post.update(tokens)

# Get the 50 most common words and their frequencies
top_words_post = word_counts_post.most_common(50)

# Plot the most common words in pre and post-training datasets
plt.figure(figsize=(20, 5))
sns.set_style("darkgrid")

# Pre-training dataset
plt.subplot(121)
sns.barplot(x=[word[0] for word in top_words_pre], y=[word[1] for word in top_words_pre])
plt.xticks(rotation=90)
plt.title("Top 50 most common words (Pre-training)")
plt.xlabel("Words")
plt.ylabel("Frequency")

# Post-training dataset
plt.subplot(122)
sns.barplot(x=[word[0] for word in top_words_post], y=[word[1] for word in top_words_post])
plt.xticks(rotation=90)
plt.title("Top 50 most common words (Post-training)")
plt.xlabel("Words")
plt.ylabel("Frequency")

plt.show()

# Create a DataFrame with the top words and their frequencies in both datasets
df_pre = pd.DataFrame(top_words_pre, columns=['word', 'frequency_pre'])
df_post = pd.DataFrame(top_words_post, columns=['word', 'frequency_post'])
df = pd.merge(df_pre, df_post, on='word', how='outer').fillna(0)

# Calculate the differences between the frequencies
df['difference'] = df['frequency_post'] - df['frequency_pre']

# Plot the differences on a bar chart
plt.figure(figsize=(20, 5))
sns.set_style("darkgrid")
sns.barplot(x='word', y='difference', data=df)
plt.xticks(rotation=90)
plt.title("Differences in word frequencies between pre and post-training datasets")
plt.xlabel("Words")
plt.ylabel("Difference in frequency")
plt.show()



NameError: ignored